# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [ ]:
reviews = pd.read_csv("reviews.csv")

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37294 entries, 0 to 37293
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       37294 non-null  int64 
 1   page             37294 non-null  int64 
 2   titre            37294 non-null  object
 3   verbatim         36314 non-null  object
 4   date             37294 non-null  object
 5   note             37294 non-null  int64 
 6   reponse          7972 non-null   object
 7   date_experience  37294 non-null  object
 8   fournisseur      37294 non-null  object
 9   source           37294 non-null  object
dtypes: int64(3), object(7)
memory usage: 2.8+ MB


In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [ ]:
# ASSIGN A LABEL TO EACH REVIEW TO DIFFERENCITATE POSITIVE AND NEGATIVE REVIEWS
reviews.loc[reviews['note'] < 4, 'Label'] = 'Negative' 
reviews.loc[reviews['note'] >= 4, 'Label'] = 'Positive' 

# REMOVE REVIEWS WITH NULL VALUES
# There is no lines with no notes (cf. cell 5)
reviews = reviews.dropna()

For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [ ]:
# Filter the reviews
reviews['len'] = reviews['verbatim'].str.split().str.len() #compute length of each review (number of words)
reviews = reviews[reviews['len']<100] # Filter on sentences with length < 100
reviews_df = reviews[:1000]

In [ ]:
#Save in two variables the comments and the labels
sentences = list(reviews_df['verbatim'].values)
labels = reviews_df['Label'].values

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
# Apply tokenization
tokenized = [tokenizer(sentence) for sentence in sentences]

In [ ]:
tokenized[0]

{'input_ids': [101, 25540, 2666, 2755, 5397, 1037, 7367, 2015, 7846, 4078, 25158, 2015, 4654, 2953, 16313, 11390, 999, 25540, 2666, 12256, 2755, 27595, 4895, 6627, 8713, 9013, 10364, 3393, 11721, 2480, 1010, 2632, 5668, 10861, 24665, 20952, 1049, 1521, 1037, 29316, 10480, 2099, 10861, 3393, 11721, 2480, 27859, 4183, 2139, 3900, 4372, 2326, 18033, 12256, 8833, 13665, 999, 2765, 4017, 4078, 5352, 16655, 2755, 5397, 2139, 6390, 19329, 2074, 2063, 10364, 15068, 19716, 4313, 3393, 4012, 13876, 11236, 11322, 3089, 4226, 2139, 12256, 25272, 8833, 13665, 1010, 5142, 4630, 3393, 11721, 2480, 2711, 2638, 9765, 6970, 8159, 2226, 7505, 5003, 15775, 21041, 7869, 1046, 1521, 9932, 4241, 3393, 4189, 2063, 2000, 4904, 7367, 5313, 999, 25540, 2666, 3828, 3372, 3653, 4859, 2890, 1048, 1521, 23157, 1037, 17153, 2094, 14736, 2015, 24110, 2094, 2006, 2102, 3393, 3126, 4654, 24759, 7413, 15451, 5369, 5397, 8557, 3672, 4606, 2711, 2638, 9765, 2474, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
len(tokenized)

1000

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [ ]:
# Perform the padding
max_len = 0
index = 0

for i in range(len(tokenized)):
    
    if len(tokenized[i]['input_ids']) > max_len:
        index = i
        max_len = len(tokenized[i]['input_ids'])

max_len

226

In [ ]:
padded = []

for i in range(len(tokenized)):
    pad = tokenized[i]['input_ids'] + [0]*(max_len - len(tokenized[i]['input_ids']))
    padded.append(pad)

Our dataset is now in the padded variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

(1000, 226)

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(np.array(padded) != 0, 1, 0)
attention_mask.shape

(1000, 226)

In [ ]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [ ]:
# Split the data in train and test with the train_test_split function

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)


## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [ ]:
parameters = {'C':np.linspace(0,50,199)}

reg = LogisticRegression()

gridcv = GridSearchCV(reg, parameters)

gridcv.fit(X_train,y_train)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([ 0.        ,  0.25252525,  0.50505051,  0.75757576,  1.01010101,
        1.26262626,  1.51515152,  1.76767677,  2.02020202,  2.27272727,
        2.52525253,  2.77777778,  3.03030303,  3.28282828,  3.53535354,
        3.78787879,  4.04040404,  4.29292929,  4.54545455,  4.7979798 ,
        5.05050505,  5.3030303 ,  5.55555556,  5.80808081,  6.06060606,
        6.31313131,  6.56565...
       41.66666667, 41.91919192, 42.17171717, 42.42424242, 42.67676768,
       42.92929293, 43.18181818, 43.43434343, 43.68686869, 43.93939394,
       44.19191919, 44.44444444, 44.6969697 , 44.94949495, 45.2020202 ,
       45.45454545, 45.70707071, 45.95959596, 46.21212121, 46.46464646,
       46.71717172, 46.96969697, 47.22222222, 47.47474747, 47.72727273,
       47.97979798, 48.23232323, 48.48484848, 48.73737374, 48.98989899,
       49.24242424, 49.49494949, 49.74747475, 50.        ])})

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [ ]:
# Fit the logistic regression model
# Using a first model using the default value
reg = LogisticRegression()

reg.fit(X_train,y_train)

#Using a second model based on the best C value found through GridSearch
reg_CV = LogisticRegression(C = gridcv.best_params_['C'])

reg_CV.fit(X_train,y_train)



LogisticRegression(C=30.050505050505052)

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
# Evaluate the score of the model

print('The score with default value of C is', reg.score(X_test,y_test))

print('The score with grid searched value of C is', reg_CV.score(X_test,y_test))

The score with default value of C is 0.9181818181818182
The score with grid searched value of C is 0.9121212121212121


Interestingly, the score obtained through cross validation is less performant on the testing set than the default value.

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

# COmpare your model with a dummy classifier
clf.fit(X_train,y_train)

clf.score(X_test,y_test)

0.5727272727272728

So our model clearly does better than a dummy classifier